# A full business solution

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

Business Applications¶
In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

In terms of techniques, this is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

In terms of business applications - generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.

In [1]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [21]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-openai-api-key')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [13]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    body: str
    links: List[str]
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [14]:
ed = Website("https://knowledgeseedsolutions.co.uk/")
print(ed.get_contents())

Webpage Title:
Delivering Transformative Change and Regulatory Compliance
Webpage Contents:
KNOWLEDGE SEED SOLUTIONS LIMITED
Delivering Transformative Change and Regulatory Compliance
Knowledge Seed Solutions is a pioneering consultancy that seamlessly brings together regulatory knowledge, industry experience and Technical knowhow to enhance operational excellence and meet regulatory compliance.
Knowledge Seed Verticals
Our comprehensive solutions and services cater to a diverse clientele, ranging from top tier Investment Banks, FinTech, RegTech, Financial Market Infrastructure Providers, and Regulatory Reporting Service Providers.
Regulatory Change Practice
Executing various Change projects to meet the timelines and demands of regulators across EMEA and UK.
Process Excellence and Process Improvement
Crafting a methodical approach to eliminate waste and improve efficiency.
Transaction Reporting
Driving regulatory compliance to meet NFRR reporting across jurisdictions covering EMIR REFI

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [15]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [16]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [17]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://knowledgeseedsolutions.co.uk/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://knowledgeseedsolutions.co.uk/
https://knowledgeseedsolutions.co.uk
https://knowledgeseedsolutions.co.uk
https://wordpress.com/log-in?redirect_to=https%3A%2F%2Fr-login.wordpress.com%2Fremote-login.php%3Faction%3Dlink%26back%3Dhttps%253A%252F%252Fknowledgeseedsolutions.co.uk%252F2023%252F11%252F18%252Fhello-world%252F
https://knowledgeseedsolutions.co.uk
https://knowledgeseedsolutions.wordpress.com/wp-admin/site-editor.php?postType=wp_template&postId=pub/twentytwentyfour//home
https://wordpress.com/start/
https://wordpress.com/log-in?redirect_to=https%3A%2F%2Fr-login.wordpress.com%2Fremote-login.php%3Faction%3Dlink%26back%3Dhttps%253A%252F%252Fknowledgeseedsolutions.co.uk%25

In [18]:
def get_links(url):
    website = Website(url)
    completion = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [22]:
get_links("https://knowledgeseedsolutions.co.uk/")

{'links': [{'type': 'home page',
   'url': 'https://knowledgeseedsolutions.co.uk/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [23]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [24]:
print(get_all_details("https://knowledgeseedsolutions.co.uk/'"))

Found links: {'links': [{'type': 'homepage', 'url': 'https://knowledgeseedsolutions.co.uk/'}]}
Landing page:
Webpage Title:
Delivering Transformative Change and Regulatory Compliance
Webpage Contents:
KNOWLEDGE SEED SOLUTIONS LIMITED
Delivering Transformative Change and Regulatory Compliance
Knowledge Seed Solutions is a pioneering consultancy that seamlessly brings together regulatory knowledge, industry experience and Technical knowhow to enhance operational excellence and meet regulatory compliance.
Knowledge Seed Verticals
Our comprehensive solutions and services cater to a diverse clientele, ranging from top tier Investment Banks, FinTech, RegTech, Financial Market Infrastructure Providers, and Regulatory Reporting Service Providers.
Regulatory Change Practice
Executing various Change projects to meet the timelines and demands of regulators across EMEA and UK.
Process Excellence and Process Improvement
Crafting a methodical approach to eliminate waste and improve efficiency.
Trans

In [25]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [26]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [27]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [28]:
create_brochure("knowledgeseedsolutions", "https://knowledgeseedsolutions.co.uk/'")

Found links: {'links': [{'type': 'company homepage', 'url': 'https://knowledgeseedsolutions.co.uk/'}]}


# Knowledge Seed Solutions Limited

## Transformative Change and Regulatory Compliance

At Knowledge Seed Solutions, we are pioneers in the consultancy space, delivering transformative change and ensuring regulatory compliance for a diverse clientele that includes top-tier investment banks, fintech firms, regtech companies, financial market infrastructure providers, and regulatory reporting service providers.

### Our Expertise

**Regulatory Change Practice**  
We execute various change projects tailored to meet the demands and timelines set by regulators across EMEA and the UK.

**Process Excellence and Improvement**  
Our approach focuses on eliminating waste and enhancing operational efficiency through methodical processes.

**Transaction Reporting**  
We drive regulatory compliance across jurisdictions, ensuring adherence to requirements such as EMIR REFIT, CFTC REWRITE, SFTR, FINFRAG, MAS, and ASIC.

**Operational Risk Management**  
Our team provides functional expertise to define, measure, analyze, and improve processes related to operational, compliance, and audit risks.

**Business Analysis and Project Management**  
We facilitate business analysis and project delivery by managing cross-functional teams worldwide.

**Target Operating Model**  
We analyze existing operations (AS IS) and design a future state (TO BE) in alignment with standards set by project sponsors and key stakeholders.

### Our Core Values

- **Integrity**
- **Accountability**
- **Empathy**
- **Trust**
- **Discipline**

### Our Guiding Principles

We believe in:
- **Innovation**
- **Collaboration**
- **Partnership**
- **Diversity**
- **Sustainability**

### Company Culture

At Knowledge Seed Solutions, our culture is rooted in our core values. We prioritize integrity and accountability while fostering an environment of empathy and trust. We encourage innovation and collaboration and embrace diversity as a strength, driving us towards sustainable success.

### Career Opportunities

We are always looking for talented individuals who share our passion for regulatory compliance and operational excellence. Whether you’re an experienced professional or just starting your career, Knowledge Seed Solutions offers a dynamic environment where you can grow and make an impact.

---

**Contact Us:**  
Email: [Contact@knowledgeseedsolutions.com](mailto:Contact@knowledgeseedsolutions.com)

Join us in delivering change and compliance with integrity and expertise!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("Anthropic", "https://anthropic.com")

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")